In [1]:
import psycopg2
from psycopg2.extras import execute_values
import random

import sql_queries as sq
from database import database 

In [2]:
# -psql -U postgres
# -(enter your password)
# -CREATE DATABASE _____;
# -\c _____;
# -(paste db.sql)

#Replace cp317_final with your db name, and postres with your username, and password with your password
database_instance = database('cp317_final','postgres','password','localhost','5432')
conn, cur = database_instance.get_instance()

database_instance.delete_all()
database_instance.create_type_genre()

#.......................


In [3]:
#TEMPORARY-TEST-DATA-UNTIL-THATS-READY
movies = [
    ["The Shawshank Redemption", "Drama", "Movie"],
    ["The Godfather", "Crime", "Movie"],
    ["The Dark Knight", "Action", "Movie"],
    ["Pulp Fiction", "Crime", "Movie"],
    ["Forrest Gump", "Drama", "Movie"],
    ["Inception", "Sci-Fi", "Movie"],
    ["Fight Club", "Drama", "Movie"],
    ["The Matrix", "Action", "Movie"],
    ["Goodfellas", "Crime", "Movie"],
    ["The Lord of the Rings: The Fellowship of the Ring", "Fantasy", "Movie"],
    ["Star Wars: Episode V - The Empire Strikes Back", "Fantasy", "Movie"],
    ["The Silence of the Lambs", "Thriller", "Movie"],
    ["Schindler's List", "History", "Movie"],
    ["The Green Mile", "Drama", "Movie"],
    ["Gladiator", "Action", "Movie"],
    ["Interstellar", "Sci-Fi", "Movie"],
    ["The Lion King", "Animation", "Movie"],
    ["Saving Private Ryan", "War", "Movie"],
    ["The Departed", "Crime", "Movie"],
    ["Django Unchained", "Western", "Movie"]]

for movie in movies:
    cur.execute(sq.get_type(), (movie[2],))
    type = cur.fetchall()[0][0]
    cur.execute(sq.get_genre(), (movie[1],))
    genre = cur.fetchall()[0][0]
    date_released = "2069-01-10"
    studio = "TESTING"
    producer = "TESTING"
    name = movie[0]
    cur.execute(sq.add_media(), (type, genre, genre, genre, date_released,studio,producer,name,))
conn.commit()


In [4]:
#Sample Real Account (Used For Presentation)

count_media = 50
max_reviews_per_account = 10

#Adding Account Names
account_names = ['Venkat Gunturi', 'Gavin Heslip', 'Hashim Jama', 
                 'Zach Reid', 'Lucas Duncan', 'John Doe', 'Jane Smith',
                 'Kylie Tan', 'Bertha Blue', 'Tiffany Stoik']

cur.executemany(sq.add_account(), [(account,"TESTING" + account, "TESTING" + account, "TESTING", "TESTING" ) for account in account_names])
conn.commit()



#Sample media
cur.execute(sq.get_random_media(count_media))
media_ids = cur.fetchall()

#Sample Account Ids
account_ids = []
for name in account_names:
    cur.execute(sq.get_matching_account(), ("TESTING" + name,))
    account_ids.append(cur.fetchall()[0][0])


#Create Sample Followings
for id in account_ids:
    follow_count = random.randint(1, len(account_ids) - 1)
    account_ids_to_follow = random.sample(account_ids, follow_count)
    for follower_id in account_ids_to_follow:
        if follower_id == id: continue
        cur.execute(sq.add_following(), (id, follower_id))
conn.commit()  

#Create Sample Reviews:
for id in media_ids:
    review_count = random.randint(1, max_reviews_per_account)
    
    account_ids_to_review = random.sample(account_ids, review_count)

    for account_id in account_ids_to_review:
        rating = random.randint(1, 10)
        cur.execute(sq.add_review(), (account_id, id, rating, 'TESTING',))
conn.commit()